In [33]:
import pandas as pd
import numpy as np
#from sklearn import train_test_split
import sklearn.metrics
import dataclasses
import lightgbm as lgb
from ray import tune
from ray.tune.schedulers import PopulationBasedTraining

In [22]:
train_processed_df = pd.read_pickle("inputs/train_processed.pkl")

In [ ]:
# search_space = {

# }

# pbt_scheduler = PopulationBasedTraining(
#         time_attr='time_total_s',
#         metric='mean_accuracy',
#         mode='max',
#         perturbation_interval=600.0,
#         hyperparam_mutations={
#             "lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
#             "alpha": lambda: random.uniform(0.0, 1.0),
#         ...
#         })
# tune.run( train_processed_df, scheduler=pbt_scheduler)

In [12]:
from sklearn.model_selection import train_test_split
X=train_processed_df.drop('diabetes_mellitus', axis=1)
y=train_processed_df.diabetes_mellitus
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=314, stratify=y)
train_set = lgb.Dataset(X_train, label=y_train)
test_set = lgb.Dataset(X_test, label=y_test)

In [17]:
def LightGBMCallback(env):
    """Assumes that `valid_0` is the target validation score."""
    _, metric, score, _ = env.evaluation_result_list[0]
    tune.report(**{metric: score})

In [35]:
def train_diabetes(config):
    #data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
    #train_processed_df = pd.read_pickle("inputs/train_processed.pkl")
    data = train_processed_df.drop('diabetes_mellitus', axis=1)
    target = train_processed_df.diabetes_mellitus
    train_x, test_x, train_y, test_y = train_test_split(
        data, target, test_size=0.25)
    train_set = lgb.Dataset(train_x, label=train_y)
    test_set = lgb.Dataset(test_x, label=test_y)
    gbm = lgb.train(
        config,
        train_set,
        valid_sets=[test_set],
        verbose_eval=False,
        callbacks=[LightGBMCallback])
    preds = gbm.predict(test_x)
    pred_labels = np.rint(preds)
    tune.report(
        mean_accuracy=sklearn.metrics.roc_auc_score(test_y, pred_labels),
        done=True)


In [36]:
config = {
        "objective": "binary",
        #"metric": "binary_error",
        "metric": "mean_accuracy",
        "verbose": -1,
        "boosting_type": tune.grid_search(["gbdt", "dart"]),
        "num_leaves": tune.randint(10, 1000),
        "learning_rate": tune.loguniform(1e-2, 1e-1)
    }
analysis = tune.run(
        train_diabetes,
        metric="auc",
        mode="min",
        config=config,
        num_samples=2, #Number of times to sample from the hyperparameter space
        #scheduler=ASHAScheduler()
        local_dir="/Users/anitaclement/Documents/Projects/internal_projects/populationBasedTraining/logging",
        scheduler=PopulationBasedTraining()
        )

print("Best hyperparameters found were: ", analysis.best_config)

2021-03-18 15:03:27,518	WARNING worker.py:1107 -- Warning: The actor ImplicitFunc has size 85149703 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.
2021-03-18 15:03:27,646	WARNING util.py:152 -- The `start_trial` operation took 0.858 s, which may be a performance bottleneck.


Trial name,status,loc,boosting_type,learning_rate,num_leaves
train_diabetes_17355_00000,RUNNING,,gbdt,0.0200876,439


2021-03-18 15:03:28,892	WARNING util.py:152 -- The `start_trial` operation took 0.679 s, which may be a performance bottleneck.
2021-03-18 15:03:30,083	WARNING util.py:152 -- The `start_trial` operation took 0.733 s, which may be a performance bottleneck.
2021-03-18 15:03:31,472	WARNING util.py:152 -- The `start_trial` operation took 0.765 s, which may be a performance bottleneck.
(pid=35848) /Users/anitaclement/Documents/Projects/internal_projects/populationBasedTraining/pbt/lib/python3.7/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
(pid=35848)   warnings.warn('Overriding the parameters from Reference Dataset.')
(pid=35848) /Users/anitaclement/Documents/Projects/internal_projects/populationBasedTraining/pbt/lib/python3.7/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
(pid=35848)   warnings.warn('{} in param dict is overridden.'.format(cat_alias))
(pid=35863) /Users/anitaclement/

nt/Documents/Projects/internal_projects/populationBasedTraining/pbt/lib/python3.7/site-packages/ray/tune/function_runner.py", line 576, in _trainable_func
(pid=35863)     output = fn()
(pid=35863)   File "<ipython-input-35-e67547cfe31f>", line 15, in train_diabetes
(pid=35863)   File "/Users/anitaclement/Documents/Projects/internal_projects/populationBasedTraining/pbt/lib/python3.7/site-packages/lightgbm/engine.py", line 267, in train
(pid=35863)     evaluation_result_list=evaluation_result_list))
(pid=35863)   File "<ipython-input-17-c4e673d45b99>", line 3, in LightGBMCallback
(pid=35863) IndexError: list index out of range
(pid=35863) Exception in thread Thread-2:
(pid=35863) Traceback (most recent call last):
(pid=35863)   File "/Users/anitaclement/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
(pid=35863)     self.run()
(pid=35863)   File "/Users/anitaclement/Documents/Projects/internal_projects/populationBasedTraining/pbt/lib/python3.7/site-packages/ray/t

TuneError: ('Trials did not complete', [train_diabetes_17355_00000, train_diabetes_17355_00001, train_diabetes_17355_00002, train_diabetes_17355_00003])

In [49]:
import lightgbm as lgb
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split

from ray import tune
from ray.tune.schedulers import ASHAScheduler


def LightGBMCallback(env):
    """Assumes that `valid_0` is the target validation score."""
    _, metric, score, _ = env.evaluation_result_list[0]
    tune.report(**{metric: score})


def train_diabetes(config):
    data = train_processed_df.drop('diabetes_mellitus', axis=1)
    target = train_processed_df.diabetes_mellitus
    #data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
    train_x, test_x, train_y, test_y = train_test_split(
        data, target, test_size=0.20)
    train_set = lgb.Dataset(train_x, label=train_y)
    test_set = lgb.Dataset(test_x, label=test_y)
    gbm = lgb.train(
        config,
        train_set,
        valid_sets=[test_set],
        verbose_eval=False,
        callbacks=[LightGBMCallback])
    preds = gbm.predict(test_x)
    pred_labels = np.rint(preds)
    tune.report(
        #mean_accuracy=sklearn.metrics.accuracy_score(test_y, pred_labels),
        #binary_error=sklearn.metrics.accuracy_score(test_y, pred_labels),
        binary_error=sklearn.metrics.roc_auc_score(test_y, pred_labels),
        done=True)


if __name__ == "__main__":
    config = {
        "objective": "binary",
        "metric": "binary_error",
        "verbose": -1,
        "boosting_type": tune.grid_search(["gbdt", "dart"]),
        "num_leaves": tune.randint(10, 40),
        'min_child_samples': tune.randint(100, 400),
        'min_child_weight': tune.grid_search([1e-7,1e-6,1e-5, 1e-4, 1e-3, 1e-2,1e-1]),
        'subsample': tune.uniform(0.4, 0.6),
        'colsample_bytree': tune.uniform(0.2, 0.6),
        'reg_alpha': tune.grid_search([0, 1, 2, 5, 7, 10, 50]),
        'reg_lambda': tune.grid_search([0, 1, 5, 10, 20, 50]),
        'scale_pos_weight': tune.grid_search([1, 2, 3]),
        #"learning_rate": tune.loguniform(1e-2, 1e-1)
        "learning_rate": 0.01
    }

    analysis = tune.run(
        train_diabetes,
        metric="binary_error",
        #mode="min",
        mode="max",
        config=config,
        num_samples=2,
        #scheduler=ASHAScheduler())
        scheduler=PopulationBasedTraining(),
        local_dir="/Users/anitaclement/Documents/Projects/internal_projects/populationBasedTraining/logging")

    print("Best hyperparameters found were: ", analysis.best_config)

2021-03-18 15:55:17,589	WARNING worker.py:1107 -- Warning: The actor ImplicitFunc has size 85149680 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.
2021-03-18 15:55:17,691	WARNING util.py:152 -- The `start_trial` operation took 1.150 s, which may be a performance bottleneck.


Trial name,status,loc,boosting_type,colsample_bytree,min_child_samples,min_child_weight,num_leaves,reg_alpha,reg_lambda,scale_pos_weight,subsample
train_diabetes_540e8_00000,RUNNING,,gbdt,0.443979,126,1e-07,15,0,0,1,0.452008


2021-03-18 15:55:19,207	WARNING util.py:152 -- The `start_trial` operation took 0.864 s, which may be a performance bottleneck.
2021-03-18 15:55:20,472	WARNING util.py:152 -- The `start_trial` operation took 0.755 s, which may be a performance bottleneck.
2021-03-18 15:55:22,893	WARNING util.py:152 -- The `start_trial` operation took 1.726 s, which may be a performance bottleneck.


Trial name,status,loc,boosting_type,colsample_bytree,min_child_samples,min_child_weight,num_leaves,reg_alpha,reg_lambda,scale_pos_weight,subsample
train_diabetes_540e8_00000,RUNNING,,gbdt,0.443979,126,1e-07,15,0,0,1,0.452008
train_diabetes_540e8_00001,RUNNING,,dart,0.304379,148,1e-07,29,0,0,1,0.522935
train_diabetes_540e8_00002,RUNNING,,gbdt,0.417973,288,1e-06,34,0,0,1,0.570295
train_diabetes_540e8_00003,RUNNING,,dart,0.262031,309,1e-06,21,0,0,1,0.543832


2021-03-18 15:55:25,049	WARNING util.py:152 -- The `start_trial` operation took 1.489 s, which may be a performance bottleneck.
2021-03-18 15:55:27,451	WARNING util.py:152 -- The `start_trial` operation took 1.436 s, which may be a performance bottleneck.
2021-03-18 15:55:29,642	WARNING util.py:152 -- The `start_trial` operation took 1.323 s, which may be a performance bottleneck.


...

Trial name,status,loc,boosting_type,colsample_bytree,min_child_samples,min_child_weight,num_leaves,reg_alpha,reg_lambda,scale_pos_weight,subsample,iter,total time (s),binary_error
train_diabetes_540e8_00053,RUNNING,192.168.0.4:36908,dart,0.390969,195,0.01,10,5,0,1,0.529137,89,46.553,0.215542
train_diabetes_540e8_00054,RUNNING,192.168.0.4:36916,gbdt,0.301228,246,0.1,20,5,0,1,0.522191,84,37.5625,0.215965
train_diabetes_540e8_00055,RUNNING,192.168.0.4:36926,dart,0.364621,297,0.1,18,5,0,1,0.411705,44,27.4304,0.216234
train_diabetes_540e8_00056,RUNNING,192.168.0.4:36937,gbdt,0.3686,132,1e-07,22,7,0,1,0.434315,35,24.8792,0.214198
train_diabetes_540e8_00057,RUNNING,192.168.0.4:36943,dart,0.200616,260,1e-07,39,7,0,1,0.494836,12,18.1568,0.216195
train_diabetes_540e8_00058,RUNNING,,gbdt,0.306115,282,1e-06,18,7,0,1,0.517775,,,
train_diabetes_540e8_00059,RUNNING,,dart,0.361985,191,1e-06,24,7,0,1,0.589797,,,
train_diabetes_540e8_00005,PAUSED,,dart,0.52967,158,1e-05,23,0,0,1,0.560506,92,60.1112,0.217233
train_diabetes_540e8_00013,PAUSED,,dart,0.482145,316,0.1,35,0,0,1,0.491346,96,60.2273,0.215312
train_diabetes_540e8_00015,PAUSED,,dart,0.572314,317,1e-07,39,1,0,1,0.48328,96,60.0299,0.216772


Result for train_diabetes_540e8_00056:
  binary_error: 0.2141979102642901
  date: 2021-03-18_16-02-00
  done: false
  experiment_id: 64d6264fa7d84740b9ad2c913ab24783
  hostname: MacBook-Pro-3
  iterations_since_restore: 36
  node_ip: 192.168.0.4
  pid: 36937
  time_since_restore: 25.108046054840088
  time_this_iter_s: 0.22885823249816895
  time_total_s: 25.108046054840088
  timestamp: 1616083320
  timesteps_since_restore: 0
  training_iteration: 36
  trial_id: 540e8_00056
  
(pid=36955) /Users/anitaclement/Documents/Projects/internal_projects/populationBasedTraining/pbt/lib/python3.7/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
(pid=36955)   warnings.warn('Overriding the parameters from Reference Dataset.')
(pid=36955) /Users/anitaclement/Documents/Projects/internal_projects/populationBasedTraining/pbt/lib/python3.7/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
(pid=36955)   war

Trial name,status,loc,boosting_type,colsample_bytree,min_child_samples,min_child_weight,num_leaves,reg_alpha,reg_lambda,scale_pos_weight,subsample,iter,total time (s),binary_error
train_diabetes_540e8_00054,RUNNING,192.168.0.4:36916,gbdt,0.301228,246,0.1,20,5,0,1,0.522191,100,41.0996,0.209242
train_diabetes_540e8_00055,RUNNING,192.168.0.4:36926,dart,0.364621,297,0.1,18,5,0,1,0.411705,60,32.2137,0.216234
train_diabetes_540e8_00056,RUNNING,192.168.0.4:36937,gbdt,0.3686,132,1e-07,22,7,0,1,0.434315,56,29.8933,0.214198
train_diabetes_540e8_00057,RUNNING,192.168.0.4:36943,dart,0.200616,260,1e-07,39,7,0,1,0.494836,35,23.0196,0.216195
train_diabetes_540e8_00058,RUNNING,192.168.0.4:36955,gbdt,0.306115,282,1e-06,18,7,0,1,0.517775,5,16.2489,0.216887
train_diabetes_540e8_00059,RUNNING,,dart,0.361985,191,1e-06,24,7,0,1,0.589797,,,
train_diabetes_540e8_00060,RUNNING,,gbdt,0.338441,192,1e-05,18,7,0,1,0.443804,,,
train_diabetes_540e8_00005,PAUSED,,dart,0.52967,158,1e-05,23,0,0,1,0.560506,92,60.1112,0.217233
train_diabetes_540e8_00013,PAUSED,,dart,0.482145,316,0.1,35,0,0,1,0.491346,96,60.2273,0.215312
train_diabetes_540e8_00015,PAUSED,,dart,0.572314,317,1e-07,39,1,0,1,0.48328,96,60.0299,0.216772


Result for train_diabetes_540e8_00056:
  binary_error: 0.2141979102642901
  date: 2021-03-18_16-02-05
  done: false
  experiment_id: 64d6264fa7d84740b9ad2c913ab24783
  hostname: MacBook-Pro-3
  iterations_since_restore: 57
  node_ip: 192.168.0.4
  pid: 36937
  time_since_restore: 30.111028909683228
  time_this_iter_s: 0.21777105331420898
  time_total_s: 30.111028909683228
  timestamp: 1616083325
  timesteps_since_restore: 0
  training_iteration: 57
  trial_id: 540e8_00056
  
Result for train_diabetes_540e8_00054:
  binary_error: 0.5226520941823881
  date: 2021-03-18_16-02-05
  done: true
  experiment_id: 5bd97ce32d8042078d7a0d4851659807
  hostname: MacBook-Pro-3
  iterations_since_restore: 101
  node_ip: 192.168.0.4
  pid: 36916
  time_since_restore: 42.8869469165802
  time_this_iter_s: 1.7873749732971191
  time_total_s: 42.8869469165802
  timestamp: 1616083325
  timesteps_since_restore: 0
  training_iteration: 101
  trial_id: 540e8_00054
  
2021-03-18 16:02:07,336	WARNING util.py:152 

KeyboardInterrupt: 